In [ ]:
import gwcs
import numpy as np
from astropy import units as u
from astropy.coordinates import ICRS
from astropy.io import fits
from astropy.modeling import models
from astropy.nddata import NDData
from astropy.wcs import WCS
from gwcs import coordinate_frames as cf

from jdaviz import Imviz
from jdaviz.configs.imviz.wcs_utils import generate_rotated_wcs

%matplotlib inline

# Plain array

In [ ]:
a = np.random.random((2048, 4096))
a[:100, :400] = 1  # Bright corner for sanity check

# Data with FITS WCS and unit

In [ ]:
w = WCS({'WCSAXES': 2,
         'CRPIX1': 2100.0,
         'CRPIX2': 1024.0,
         'PC1_1': -1.14852e-05,
         'PC1_2': 7.01477e-06,
         'PC2_1': 7.75765e-06,
         'PC2_2': 1.20927e-05,
         'CDELT1': 1.0,
         'CDELT2': 1.0,
         'CUNIT1': 'deg',
         'CUNIT2': 'deg',
         'CTYPE1': 'RA---TAN',
         'CTYPE2': 'DEC--TAN',
         'CRVAL1': 3.581704851882,
         'CRVAL2': -30.39197867265,
         'LONPOLE': 180.0,
         'LATPOLE': -30.39197867265,
         'MJDREF': 0.0,
         'RADESYS': 'ICRS'})
hdu = fits.ImageHDU(a, name='SCI')
hdu.header.update(w.to_header())
hdu.header['BUNIT'] = 'electron/s'

In [ ]:
w.to_header()

# Data with GWCS (no unit)

In [ ]:
shift_by_crpix = models.Shift(-(2048 - 1) * u.pix) & models.Shift(-(1024 - 1) * u.pix)
matrix = np.array([[1.290551569736E-05, 5.9525007864732E-06],
                   [5.0226382102765E-06, -1.2644844123757E-05]])
rotation = models.AffineTransformation2D(matrix * u.deg, translation=[0, 0] * u.deg)
rotation.input_units_equivalencies = {"x": u.pixel_scale(1 * (u.deg / u.pix)),
                                      "y": u.pixel_scale(1 * (u.deg / u.pix))}
rotation.inverse = models.AffineTransformation2D(np.linalg.inv(matrix) * u.pix,
                                                 translation=[0, 0] * u.pix)
rotation.inverse.input_units_equivalencies = {"x": u.pixel_scale(1 * (u.pix / u.deg)),
                                              "y": u.pixel_scale(1 * (u.pix / u.deg))}
tan = models.Pix2Sky_TAN()
celestial_rotation = models.RotateNative2Celestial(
    3.581704851882 * u.deg, -30.39197867265 * u.deg, 180 * u.deg)
det2sky = shift_by_crpix | rotation | tan | celestial_rotation
det2sky.name = "linear_transform"
detector_frame = cf.Frame2D(name="detector", axes_names=("x", "y"), unit=(u.pix, u.pix))
sky_frame = cf.CelestialFrame(reference_frame=ICRS(), name='icrs', unit=(u.deg, u.deg))
pipeline = [(detector_frame, det2sky), (sky_frame, None)]
w2 = gwcs.WCS(pipeline)

In [ ]:
ndd_gwcs = NDData(a, wcs=w2)

This next cell does not work, see https://github.com/spacetelescope/gwcs/issues/408

# Fake data with desired orientation

This is no longer needed with the Simple Image Rotation plugin.

# Show them in Imviz

In [ ]:
imviz = Imviz(verbosity='warning')

In [ ]:
imviz.load_data(hdu, data_label='jb5g05ubq_flt')

In [ ]:
imviz.load_data(ndd_gwcs, data_label='gwcs')

In [ ]:
imviz.load_data(a, data_label='no_wcs')

In [ ]:
imviz.app

# Back to basics

This shows the low-level function to produce a WCS with desired rotation angle.

In [ ]:
from jdaviz.configs.imviz.wcs_utils import draw_compass_mpl

In [ ]:
aa = np.random.random((2, 2))
w1 = generate_rotated_wcs(0)
draw_compass_mpl(aa, wcs=w1);

In [ ]:
w2 = generate_rotated_wcs(-90)
draw_compass_mpl(aa, wcs=w2);